# Gensim 패키지
- Python으로 작성된 오픈 소스 라이브러리로, 자연어 처리와 관련된 다양한 기능을 제공한다.
- 주요 기능
    - **Word Embeddings**
        - word2vec, fastext, doc2vec 등 다양한 word embedding 모델을 제공
    - **토픽 모델링 (Topic Modeling)**
        - LDA등 문장의 주제를 파악하는 모델 제공
    - **텍스트/word 유사도 계산**
    - **문서 군집화**
        - 비슷한 주제의 문서들을 군집화.
    - 다양한 dataset과 pretrained model 제공
        - https://github.com/piskvorky/gensim-data
- https://radimrehurek.com/gensim/

## 설치
- `pip install gensim`

# Word2Vec 학습

- gensim.models.Word2Vec
- 주요 파라미터
    - sentences
        -  학습에 사용할 문서의 리스트. 각 문서의 단어들을 리스트로 묶고 그 문서들을 리스트로 묶은 중첩 리스트.
        - 예시: \[\['word1', 'word2', 'word3'], \['word4', 'word5']]
    - vector_size
        -  embedding vector 크기. 기본값: 100
    - window
        -  context window 크기. 중심단어를 기준으로 좌우 몇개의 단어를 확인하는지 크기. 기본값: 5
    - min_count
        - 이 설정보다 낮은 빈도로 등장하는 단어는 무시한다. 데이터 노이즈를 줄이는데 도움이된다. 기본값: 5
    - sg
        - 모델 아키텍처 결정.
        - `0`: CBOW, `1`: Skip-gram. 기본값: 0
    - epochs
        - epochs 수 설정. 기본값: 5
    - alpha
        - initial leaning rate. 기본값: 0.025
    - min_alpha
        - 최소 learning rate. 기본값: 0.0001
        - epoch 마다 learning rate를 alpha 에서 min_alpha 까지 선형적으로 줄여나간다.
    - workers
        -  사용 Thread 수. 기본값: 3

## 학습(Train)
1. Word2Vec 의 initializer에 sentences를 넣어 한번에 학습한다.
2. Word2Vec 클래스에 학습 설정을 하고 `train()` 메소드를 이용해 학습한다.
    - epoch 단위로 작업을 할 경우 사용

In [25]:
# 샘플 텍스트 데이터
sentences = [
    "Natural language processing is an exciting field of study",
    "Word embeddings are a type of word representation",
    "Gensim is a powerful library for text processing",
    "Word2Vec creates vector representations of words", 
    "Gensim runs on Linux, Windows and OS X, as well as any other platform that supports Python and NumPy."
    "All Gensim source code is hosted on Github under the GNU LGPL license, maintained by its open source community.",
    "For commercial arrangements, see Business Support.",
    "Gensim can process arbitrarily large corpora, using data-streamed algorithms.",
    "There are no \"dataset must fit in RAM\" limitations."
]

In [26]:
import re
import nltk
def tokenizer(docs):
    # 소문자로 모두 변환
    # 알파벳, 숫자, _를 제외한 모든 문자들을 제거
    return [nltk.word_tokenize(re.sub(r"[^\w\s]", "", doc.lower())) for doc in docs]
    # [^\w] : \w(알파벳,숫자,_), \s(공백)을 제외한 나머지 문자 찾기 / \w와 \s 사이에 or의 의미 존재

In [27]:
tokens = tokenizer(sentences)

In [28]:
import os
os.cpu_count()

8

In [29]:
# train

from gensim.models import Word2Vec
import os
model1 = Word2Vec(
    sentences=tokens,                # 학습 시킬 데이터
    vector_size=10,                  # embedding vector의 차원(한개 단어에서 몇개의 feature를 추출할지)
    window=2,                        # window size 설정. 주변단어의 개수
    min_count=1,                     # 최소 출연 빈도수
    epochs=10,
    workers=os.cpu_count()           # 병렬처리 개수
)

In [30]:
# 학습데이터, epoch을 설정하지 않음 => 학습 안된 모델을 반환
model2 = Word2Vec(vector_size=10, window=2, min_count=1, workers=os.cpu_count())
# model에 vocab 설정
model2.build_vocab(tokens)
# 학습
epoch=10
for e in range(epoch):
    model2.train(
        tokens,         # 학습데이터
        total_examples=model2.corpus_count,         # 학습데이터(문서) 개수
        epochs=1,
        compute_loss=True                           # 학습이 끝나면 loss를 계산
    )
    loss = model2.get_latest_training_loss()        # train() 시 계산된 loss 조회
    print(f"{e} epoch loss:{loss}")

0 epoch loss:147.74391174316406
1 epoch loss:139.40313720703125
2 epoch loss:108.19081115722656
3 epoch loss:159.9152069091797
4 epoch loss:163.42904663085938
5 epoch loss:143.5673370361328
6 epoch loss:136.47251892089844
7 epoch loss:130.15609741210938
8 epoch loss:164.7725067138672
9 epoch loss:156.24295043945312


## 학습 후 결과 조회

- **KeyedVectors 조회**
    - KeyedVectors는 단어와 vector를 매핑한 객체로 embedding vector를 이용한 다양한 조회를 지원한다.
    - model.wv 로 조회해서 사용.
- **Embedding Vector 조회**
  - model.wv.vectors
- **단어 목록 조회**
    - model.wv.index_to_key, model.wv.key_to_index
- **단어 벡터 조회**
    - model.wv[word]: 특정 단어의 vector반환
- **Vocab에 대상 단어가 있는지 확인**
    - "대상단어" in model.wv
- **유사단어들 찾기**
    - model.wv.most_similar(word)
- **단어간 유사도 비교**
    - model.wv.similarity(word1, word2)
- 유사도를 계산할 때 **코사인 유사도(Cosine Similarity)** 를 사용한다.

> # 코사인 유사도
> - 두 벡터 간의 유사성을 측정하는 중요한 방법 중 하나.
> - 코사인 유사도는 두 벡터 간의 코사인 각도를 이용하여 유사도를 계산한다. 이때 벡터의 **크기는 결과에 영향을 미치지 않고, 오직 방향만이 중요**하다.
> ## 공식
> 
> $$ similarity = cos(\theta) = \frac{A⋅B}{||A||\ ||B||} = \frac{\sum_{i=1}^{n}{A_i×B_i}}{\sqrt{\sum_{i=1}^{n}(A_i)^2}×\sqrt{\sum_{i=1}^{n}(B_i)^2}} $$
> 
> ## 결과 해석
> 
> - **값의 범위**: -1에서 1 사이의 값을 가집니다
>   - 1: 두 벡터가 완전히 동일한 방향 (0도의 cosine 값)
>   - 0: 두 벡터가 직교 (90도의 cosine 값)
>   - -1: 두 벡터가 정반대 방향 (180도의 cosine 값)
> 
> ![cosine_similarity](figures/gensim_consin_sim.png)
>
> ## Python 코사인 유사도 계산
> ```python
> from numpy import dot
> from numpy.linalg import norm
> 
> def cosine_similarity(A, B):
>     return dot(A, B)/(norm(A)*norm(B))
> ```



In [31]:
model1.wv               # KeyedVectors -> 토큰-embedding vector

In [32]:
model1.wv.index_to_key          # vocab: token_id -> token(단어)

['gensim',
 'is',
 'of',
 'for',
 'processing',
 'source',
 'as',
 'on',
 'word',
 'are',
 'a',
 'and',
 'words',
 'runs',
 'linux',
 'os',
 'windows',
 'vector',
 'x',
 'well',
 'any',
 'representations',
 'limitations',
 'creates',
 'word2vec',
 'text',
 'platform',
 'library',
 'powerful',
 'representation',
 'type',
 'embeddings',
 'study',
 'field',
 'exciting',
 'an',
 'language',
 'other',
 'that',
 'ram',
 'see',
 'support',
 'can',
 'process',
 'arbitrarily',
 'large',
 'corpora',
 'using',
 'datastreamed',
 'algorithms',
 'there',
 'no',
 'dataset',
 'must',
 'fit',
 'in',
 'business',
 'arrangements',
 'supports',
 'commercial',
 'python',
 'numpyall',
 'code',
 'hosted',
 'github',
 'under',
 'the',
 'gnu',
 'lgpl',
 'license',
 'maintained',
 'by',
 'its',
 'open',
 'community',
 'natural']

In [33]:
model1.wv.key_to_index

{'gensim': 0,
 'is': 1,
 'of': 2,
 'for': 3,
 'processing': 4,
 'source': 5,
 'as': 6,
 'on': 7,
 'word': 8,
 'are': 9,
 'a': 10,
 'and': 11,
 'words': 12,
 'runs': 13,
 'linux': 14,
 'os': 15,
 'windows': 16,
 'vector': 17,
 'x': 18,
 'well': 19,
 'any': 20,
 'representations': 21,
 'limitations': 22,
 'creates': 23,
 'word2vec': 24,
 'text': 25,
 'platform': 26,
 'library': 27,
 'powerful': 28,
 'representation': 29,
 'type': 30,
 'embeddings': 31,
 'study': 32,
 'field': 33,
 'exciting': 34,
 'an': 35,
 'language': 36,
 'other': 37,
 'that': 38,
 'ram': 39,
 'see': 40,
 'support': 41,
 'can': 42,
 'process': 43,
 'arbitrarily': 44,
 'large': 45,
 'corpora': 46,
 'using': 47,
 'datastreamed': 48,
 'algorithms': 49,
 'there': 50,
 'no': 51,
 'dataset': 52,
 'must': 53,
 'fit': 54,
 'in': 55,
 'business': 56,
 'arrangements': 57,
 'supports': 58,
 'commercial': 59,
 'python': 60,
 'numpyall': 61,
 'code': 62,
 'hosted': 63,
 'github': 64,
 'under': 65,
 'the': 66,
 'gnu': 67,
 'lgpl': 

In [34]:
# 특정 단어의 embedding vector 조회
model1.wv["gensim"]

array([-0.00530078,  0.00215998,  0.0511257 ,  0.08994552, -0.0929675 ,
       -0.07124555,  0.06510045,  0.08978292, -0.05092729, -0.0380263 ],
      dtype=float32)

In [35]:
# model1.wv["안녕"]           # 값이 없어 key error 발생
"인녕" in model1.wv, "gensim" in model1.wv

(False, True)

In [36]:
word = "안녕"
if word in model1.wv:
    print(model1.wv[word])
else:   print("없어 뭐 꼬아?")

없어 뭐 꼬아?


In [37]:
# 유사도 검사
model1.wv.most_similar("gensim")                    # gensim과 가장 유사한 단어를 순서대로 10개 반환
# (단어, 유사도) 유사도 -1~1  ->   1에 가까울수록 유사도가 높음 / 코사인 유사도 활용

[('numpyall', 0.7189714908599854),
 ('study', 0.713896632194519),
 ('that', 0.6710396409034729),
 ('can', 0.641579270362854),
 ('datastreamed', 0.5988761782646179),
 ('is', 0.5452057123184204),
 ('other', 0.5355265140533447),
 ('maintained', 0.5195195078849792),
 ('representations', 0.5130788087844849),
 ('in', 0.5079874396324158)]

In [38]:
model1.wv.most_similar("gensim", topn=3) 

[('numpyall', 0.7189714908599854),
 ('study', 0.713896632194519),
 ('that', 0.6710396409034729)]

In [39]:
# 단어간의 유사도
model1.wv.similarity("gensim", "study")

0.7138967

## 모델 저장 및 로딩

### 모델 저장, 로딩
- `model.save('저장파일 경로')`
  - gensim 자체 포맷으로 저장된다.
- `gensim.models.Word2Vec.load('저장파일 경로')`
  - `model.save()`로 저장된 모델을 Loading한다.

In [41]:
import os
os.makedirs("saved_models", exist_ok=True)
model1.save("saved_models/w2v_model.model")

In [42]:
from gensim.models import Word2Vec
load_model = Word2Vec.load("saved_models/w2v_model.model")

In [43]:
load_model.wv.most_similar("gensim")

[('numpyall', 0.7189714908599854),
 ('study', 0.713896632194519),
 ('that', 0.6710396409034729),
 ('can', 0.641579270362854),
 ('datastreamed', 0.5988761782646179),
 ('is', 0.5452057123184204),
 ('other', 0.5355265140533447),
 ('maintained', 0.5195195078849792),
 ('representations', 0.5130788087844849),
 ('in', 0.5079874396324158)]

### Word Embedding Vector만 저장 및 로드
- `KeyedVectors` 를 이용해 저장한다.
    - `model.wv.save_word2vec_format('저장경로', binary=True|False)`
        - binary=True: binary 파일로 저장한다. 용량이 작은 대신 내용확인이 안된다.
        - binary=False: csv(공백구분자) 형식 text로 저장한다. 내용을 확인할 수있지만 파일용량이 크다.
- `KeyedVectors.load_word2vec_format("저장경로", binary=True|False)`
    - 저장시 binary에 맞춰 읽는다.

In [44]:
model1.wv.save_word2vec_format("saved_models/keyedvector.bin", binary=True)
model1.wv.save_word2vec_format("saved_models/keyedvector.csv", binary=False)

In [47]:
from gensim.models import KeyedVectors
load_wv_bin = KeyedVectors.load_word2vec_format("saved_models/keyedvector.bin", binary=True)

In [48]:
load_wv_csv = KeyedVectors.load_word2vec_format("saved_models/keyedvector.csv", binary=False)

In [49]:
load_wv_bin.most_similar("gensim")
load_wv_csv.most_similar("gensim")

[('numpyall', 0.7189714908599854),
 ('study', 0.713896632194519),
 ('that', 0.6710396409034729),
 ('can', 0.641579270362854),
 ('datastreamed', 0.5988761782646179),
 ('is', 0.5452057123184204),
 ('other', 0.5355265140533447),
 ('maintained', 0.5195195078849792),
 ('representations', 0.5130788087844849),
 ('in', 0.5079874396324158)]

## Pretrained 모델 사용하기
- https://github.com/Kyubyong/wordvectors

In [50]:
from zipfile import ZipFile
with ZipFile("ko_new.zip") as zf:       # 압축을 풀 파일의 경로
    zf.extractall("saved_models")


In [51]:
ko_wv = KeyedVectors.load_word2vec_format(
    "saved_models/ko_new.txt",
    binary=False,
    encoding='utf-8'
)

In [53]:
word = "저녁"
result = ko_wv.most_similar(word, topn=3)
print(result)

[('아침', 0.8432559370994568), ('밤', 0.7863156795501709), ('새벽', 0.7520349025726318)]


In [54]:
ko_wv[word]

array([-1.4055068e+00, -1.1325349e+00,  8.1074381e-01, -5.0247613e-02,
        2.1491051e-01, -1.9841902e+00,  8.0733681e-01,  2.0333993e+00,
       -2.9718000e-01, -3.9601573e-01,  2.0054409e-01,  1.0981818e-01,
        6.5035206e-01,  7.2890747e-01, -1.1681221e+00, -1.8119998e+00,
        8.8641578e-01,  1.2009319e+00, -4.7502378e-01,  4.1354772e-01,
       -7.5538319e-01, -9.7632490e-02,  5.9691077e-01, -5.7045072e-01,
       -1.1796158e-01, -8.1746149e-01,  4.7462374e-01, -4.5166805e-01,
       -5.7070792e-01, -9.3876463e-01,  2.5449281e+00, -1.1985859e+00,
       -8.1546649e-02,  5.5710924e-01, -2.4425223e-01,  4.1034997e-01,
       -9.3672651e-01,  2.3033054e+00, -3.6426643e-01,  9.6841204e-01,
        3.2277573e-02, -4.0561062e-01, -7.6284319e-01, -8.6872977e-01,
       -5.5202201e-02,  1.1278684e-01, -1.9026634e+00, -1.0162231e+00,
        2.3134122e+00,  1.5543437e+00,  8.2596488e-02,  8.8935697e-01,
        6.8150657e-01, -1.5668654e+00, -9.2090231e-01, -4.5446405e-01,
      

# 빅카인즈 뉴스 데이터를 이용한 Word2Vec 학습
- 빅카인즈
    - 한국언론진흥재단에서 운영하는 뉴스빅데이터 분석 서비스 사이트
- 빅카인즈에서 특정 분야의 기사들을 수집해서 학습시킨다.
    - https://www.bigkinds.or.kr/
    - 회원가입 (구글, 네이버, 카카오 계정으로 가입 가능) 후 로그인
    - 뉴스분석 > 뉴스검색$\cdot$분석 클릭
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds1.png)
    - 기간, 언론사, 분류, 상세검색 등 검색 조건입력 후 조회
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds2.png)
    - 결과 다운로드
        - step3 분석결과및 시각화 -> 맨 아래 `엑셀다운로드` 클릭 
    ![word2vec_bigkinds1.png](figures/word2vec_bigkinds3.png)

In [56]:
import pandas as pd

df = pd.read_excel("NewsResult_20250223-20250523.xlsx")

/Users/giwonjun/anaconda3/envs/dl/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [59]:
df.shape

(2303, 19)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2303 entries, 0 to 2302
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   뉴스 식별자             2303 non-null   float64
 1   일자                 2303 non-null   int64  
 2   언론사                2303 non-null   object 
 3   기고자                1910 non-null   object 
 4   제목                 2303 non-null   object 
 5   통합 분류1             2303 non-null   object 
 6   통합 분류2             1933 non-null   object 
 7   통합 분류3             1664 non-null   object 
 8   사건/사고 분류1          2303 non-null   object 
 9   사건/사고 분류2          938 non-null    object 
 10  사건/사고 분류3          307 non-null    object 
 11  인물                 1390 non-null   object 
 12  위치                 2254 non-null   object 
 13  기관                 2247 non-null   object 
 14  키워드                2303 non-null   object 
 15  특성추출(가중치순 상위 50개)  2303 non-null   object 
 16  본문                 2303 

In [62]:
df = df[["제목", "본문"]]
df.head()

,제목,본문
0,검사 판사가 유권자 결정 바꾸는 선거법 ‘삭제→부활→삭제’ 30년사,공직선거법 위반 혐의로 재판을 받는 이재명 더불어민주당 대통령 후보는 대법원 전원합...
1,환경장관 만난 김태흠 충남지사 “청양 부여 지천댐 조속 건설을”,김태흠 충남지사가 청양 부여 지천댐 건설을 조속하게 추진해 줄 것을 정부에 요청했다...
2,[유석재의 돌발史전] 나는 이제 더 이상 ‘마오’를 존경하지 않는다,"일반적인 신문기사라면야 그럴 일이 거의 없겠습니다만, 이 ‘돌발史전’은 인터넷상에만..."
3,“이 정책만은 정권 초월해야” 부처별 ‘지속 과제’ 지키기 안간힘,"“누가 대통령 되든 정책 기조 달라져” \n기재부, 대선 후보 경제공약 분석 중 \..."
4,안전사고 대국 中 잇따라 인명 사고 발생,아시아투데이 홍순도 베이징 특파원 = 안전사고 대국이 될 수밖에 없는 중국에 최근 ...


In [ ]:
## 문서 : 개별 뉴스 기사(제목+본문)

# 전처리 + 토큰화 - 소문자로 통일 / 형태소 단위로 토큰화 진행

# gensim을 이용해 enbedding 모델 학습

# 결과 확인 : 유사도 검사